In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop, Adam
from utils.dataset import DataSet, LABELS
from models.textrnn import CuDNNGRULast as TextRNN
from models.sentiment_base import SCompositeModel as CompositeTextRNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs_sw', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'textrnn', 'cudnngrulast_test')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

MODEL_PRE = 'cudnngrulast_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
FACTOR = 0.2

VECTOR_DIM = 300
EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem_%d.pkl' % VECTOR_DIM)
RESULT_CSV = os.path.join(RESULTS_PATH, 'cudnngrulast_sw.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(TextRNN, {"max_len":train_x.shape[1], 
                    'embedding': embedding,
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeTextRNN(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop,
               factor = FACTOR)

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 281s 3ms/step - loss: 0.3661 - acc: 0.8847 - val_loss: 0.1872 - val_acc: 0.9469

Epoch 00001: _f1_monitor improved from -inf to 0.59790, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 2/100
105000/105000 [==============================] - 287s 3ms/step - loss: 0.2052 - acc: 0.9410 - val_loss: 0.1724 - val_acc: 0.9491


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00002: _f1_monitor improved from 0.59790 to 0.61921, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 3/100
105000/105000 [==============================] - 285s 3ms/step - loss: 0.1934 - acc: 0.9442 - val_loss: 0.1748 - val_acc: 0.9477

Epoch 00003: _f1_monitor improved from 0.61921 to 0.63453, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 4/100
105000/105000 [==============================] - 285s 3ms/step - loss: 0.1868 - acc: 0.9456 - val_loss: 0.1716 - val_acc: 0.9489

Epoch 00004: _f1_monitor improved from 0.63453 to 0.63575, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 5/100
105000/105000 [==============================] - 285s 3ms/step - loss: 0.1822 - acc: 0.9469 - val_loss: 0.1747 - val_acc: 0.9495

Epoch 00005: _f1_moni


Epoch 00008: _f1_monitor did not improve from 0.71044

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 9/100
105000/105000 [==============================] - 286s 3ms/step - loss: 0.2238 - acc: 0.9270 - val_loss: 0.2591 - val_acc: 0.9221

Epoch 00009: _f1_monitor improved from 0.71044 to 0.71478, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_2.h5
Epoch 10/100
105000/105000 [==============================] - 287s 3ms/step - loss: 0.2184 - acc: 0.9299 - val_loss: 0.2585 - val_acc: 0.9210

Epoch 00010: _f1_monitor improved from 0.71478 to 0.71853, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_2.h5
Epoch 11/100
105000/105000 [==============================] - 287s 3ms/step - loss: 0.2131 - acc: 0.9309 - val_loss: 0.2604 - val_acc: 0.9206

Epoch 00011: _f1_monitor did not improve from 0.71853

Epoch 00011:

105000/105000 [==============================] - 289s 3ms/step - loss: 0.0784 - acc: 0.9764 - val_loss: 0.0814 - val_acc: 0.9739

Epoch 00004: _f1_monitor improved from 0.68941 to 0.73902, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_5.h5
Epoch 5/100
105000/105000 [==============================] - 287s 3ms/step - loss: 0.0750 - acc: 0.9782 - val_loss: 0.0776 - val_acc: 0.9761

Epoch 00005: _f1_monitor improved from 0.73902 to 0.74606, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_5.h5
Epoch 6/100
105000/105000 [==============================] - 289s 3ms/step - loss: 0.0716 - acc: 0.9785 - val_loss: 0.0782 - val_acc: 0.9770

Epoch 00006: _f1_monitor did not improve from 0.74606
Epoch 7/100
105000/105000 [==============================] - 288s 3ms/step - loss: 0.0694 - acc: 0.9797 - val_loss: 0.0809 - val_acc: 0.9776

Epoch 00007: _f1_monit

105000/105000 [==============================] - 285s 3ms/step - loss: 0.4089 - acc: 0.8491 - val_loss: 0.5497 - val_acc: 0.7941

Epoch 00010: _f1_monitor did not improve from 0.76637
Epoch 11/100
105000/105000 [==============================] - 286s 3ms/step - loss: 0.3980 - acc: 0.8548 - val_loss: 0.5592 - val_acc: 0.7913

Epoch 00011: _f1_monitor did not improve from 0.76637

Epoch 00011: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 7th grain: f1_score - 0.7663657498839828, acc - 0.8028
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 288s 3ms/step - loss: 0.4821 - acc: 0.8337 - val_loss: 0.3427 - val_acc: 0.8834

Epoch 00001: _f1_monitor improved from -inf to 0.67148, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_8.h5
Epoch 2/100
105000/105000 [==============================] - 288s 3ms/step - loss: 0.3431 - acc: 0.8779 

Epoch 9/100
105000/105000 [==============================] - 287s 3ms/step - loss: 0.3244 - acc: 0.8875 - val_loss: 0.3850 - val_acc: 0.8647

Epoch 00009: _f1_monitor did not improve from 0.70864
Epoch 10/100
105000/105000 [==============================] - 287s 3ms/step - loss: 0.3183 - acc: 0.8893 - val_loss: 0.3926 - val_acc: 0.8620

Epoch 00010: _f1_monitor did not improve from 0.70864
Epoch 11/100
105000/105000 [==============================] - 285s 3ms/step - loss: 0.3107 - acc: 0.8925 - val_loss: 0.3964 - val_acc: 0.8621

Epoch 00011: _f1_monitor did not improve from 0.70864

Epoch 00011: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 12/100
105000/105000 [==============================] - 286s 3ms/step - loss: 0.3035 - acc: 0.8958 - val_loss: 0.3945 - val_acc: 0.8623

Epoch 00012: _f1_monitor did not improve from 0.70864
Epoch 13/100
105000/105000 [==============================] - 286s 3ms/step - loss: 0.3007 - acc: 0.8961 - val_loss: 0.3974 - val_a

105000/105000 [==============================] - 286s 3ms/step - loss: 0.3785 - acc: 0.8675 - val_loss: 0.3621 - val_acc: 0.8718

Epoch 00004: _f1_monitor improved from 0.72290 to 0.73427, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_13.h5
Epoch 5/100
105000/105000 [==============================] - 286s 3ms/step - loss: 0.3676 - acc: 0.8696 - val_loss: 0.3535 - val_acc: 0.8759

Epoch 00005: _f1_monitor improved from 0.73427 to 0.73964, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_13.h5
Epoch 6/100
105000/105000 [==============================] - 284s 3ms/step - loss: 0.3597 - acc: 0.8730 - val_loss: 0.3640 - val_acc: 0.8743

Epoch 00006: _f1_monitor did not improve from 0.73964
Epoch 7/100
105000/105000 [==============================] - 284s 3ms/step - loss: 0.3514 - acc: 0.8757 - val_loss: 0.3605 - val_acc: 0.8753

Epoch 00007: _f1_mon

105000/105000 [==============================] - 285s 3ms/step - loss: 0.5575 - acc: 0.8020 - val_loss: 0.5402 - val_acc: 0.8069

Epoch 00002: _f1_monitor improved from 0.51854 to 0.53049, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_16.h5
Epoch 3/100
105000/105000 [==============================] - 286s 3ms/step - loss: 0.5380 - acc: 0.8071 - val_loss: 0.5317 - val_acc: 0.8084

Epoch 00003: _f1_monitor improved from 0.53049 to 0.55807, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_16.h5
Epoch 4/100
105000/105000 [==============================] - 287s 3ms/step - loss: 0.5221 - acc: 0.8116 - val_loss: 0.5255 - val_acc: 0.8097

Epoch 00004: _f1_monitor did not improve from 0.55807
Epoch 5/100
105000/105000 [==============================] - 284s 3ms/step - loss: 0.5098 - acc: 0.8154 - val_loss: 0.5336 - val_acc: 0.8095

Epoch 00005: _f1_mon


Epoch 00002: _f1_monitor improved from 0.65926 to 0.68375, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_19.h5
Epoch 3/100
105000/105000 [==============================] - 282s 3ms/step - loss: 0.3595 - acc: 0.8696 - val_loss: 0.3411 - val_acc: 0.8753

Epoch 00003: _f1_monitor improved from 0.68375 to 0.68476, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_19.h5
Epoch 4/100
105000/105000 [==============================] - 284s 3ms/step - loss: 0.3475 - acc: 0.8744 - val_loss: 0.3419 - val_acc: 0.8734

Epoch 00004: _f1_monitor improved from 0.68476 to 0.69006, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_19.h5
Epoch 5/100
105000/105000 [==============================] - 284s 3ms/step - loss: 0.3362 - acc: 0.8799 - val_loss: 0.3355 - val_acc: 0.8761

Epoch 00005: _f1_m

In [7]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [9]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.6612534586720259
The 1th f1: 0.5310201000273408
The 2th f1: 0.7185304731808975
The 3th f1: 0.6531599202903229
The 4th f1: 0.7920485988560367
The 5th f1: 0.746057326516547
The 6th f1: 0.7458045963470159
The 7th f1: 0.7663657498839828
The 8th f1: 0.7005087468397826
The 9th f1: 0.6786577191044443
The 10th f1: 0.7086421752184826
The 11th f1: 0.7459538349100434
The 12th f1: 0.7421874466315878
The 13th f1: 0.7416624300195035
The 14th f1: 0.6994730586874767
The 15th f1: 0.7198528870968385
The 16th f1: 0.5700092483045729
The 17th f1: 0.7382701575737929
The 18th f1: 0.5752362050486297
The 19th f1: 0.712497961846517
The average f1 of val is 0.697359604752792


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)

In [12]:
print("Hello world")

Hello world
